In [1]:
from jsonobject import *
import numpy as np 
import random

In [2]:
random.seed(2021)

# Classes

In [3]:
# Convolution
class Convolution(JsonObject):
    kernel = IntegerProperty()
    padding = StringProperty()
    stride = IntegerProperty()
    nb_filter = IntegerProperty()
    fct_activation = StringProperty()

In [4]:
# Input 
class InputLayer(JsonObject):
    shape = ListProperty(int)
    #shape = StringProperty()

In [5]:
# Pooling Avg/Max
class Pooling(JsonObject):
    op = StringProperty()
    kernel = IntegerProperty(default=2)
    padding = StringProperty(default="'valid'")
    stride = IntegerProperty(default=None)

In [6]:
# Class Flatten
class Flatten(JsonObject):
    pass

In [7]:
# Dense --> Fully connected layer
class Dense(JsonObject):
    nb_neurones =  IntegerProperty()
    fct_activation = StringProperty()

# Test

In [8]:
conv1 = Convolution(kernel=7, padding="valid", stride=2, nb_filter=6, fct_activation="tanh")
conv1.to_json()

{'kernel': 7,
 'padding': 'valid',
 'stride': 2,
 'nb_filter': 6,
 'fct_activation': 'tanh'}

In [9]:
inputL = InputLayer(shape=[28,28,1])
inputL.to_json()

{'shape': [28, 28, 1]}

In [10]:
pool1 = Pooling(op = "avg")
pool1.to_json()

{'op': 'avg', 'kernel': 2, 'padding': "'valid'", 'stride': None}

# Values

In [11]:
layers_name = [Convolution, InputLayer, Pooling, Flatten, Dense]

# Table of hyperparameter value
kernel_value = [1, 2, 3, 5, 7]
stride_value = [1, 2, 3, 4, 5]
padding_value = ["valid", "same"]
#nb_filter_value = [6, 16, 64, 128, 256, 512, 1024, 2048]
fct_activation_value = ["tanh", "relu", "selu"]
nb_neurones_value = [1, 10, 84, 120, 256]
op_value = ['avg',"max"]

# Functions

In [12]:
def stop(size_archi, x):
    prob = x*size_archi
    if ( prob < random.randrange(101)):
        return True
    else : return False

In [13]:
def create_json_file(archi, file_name):
    directory = 'architecture_json/'
    
    # reset file
    archi_file = open(directory+file_name, "w")
    archi_file.close()
    
    # create file
    
    archi_file = open(directory+file_name, "a") # Open file in writting (a --> append)
    archi_file.write("""[
    """)
    archi_size = len(archi)
    i = 0
    for l in archi:

        str_layer = """\t{
            'class':'"""
        str_layer += l.__class__.__name__
        str_layer +="""',\n\t\t\t'parameters':"""
        str_layer += str(l.to_json())
        str_layer += """\n\t\t}"""
        if(i < archi_size-1):
            str_layer += ""","""
            i+=1
        str_layer = str_layer.replace("'","\"")
        
        archi_file.write(str_layer)
        
    archi_file.write("""\n]""")
    archi_file.close()

In [14]:
# compute the output 
# valid vs same
def calcul_output(input_size, l):
    output_size = 0
    if(l.padding == "valid"):
        kernel = l.kernel
        stride = l.stride
        while(input_size>=kernel):
            input_size -= stride
            output_size += 1   
    else:
        stride = l.stride
        output_size = int(input_size/stride)
    return output_size

In [15]:
# add layer to architecture 

def addLayer(archi, layer):
    input_size = archi[0].shape[0]
    feature_extra = archi[1:]
    
    # add layer if the architecture is empty
    if(feature_extra == []):
        archi.append(layer)
        return 1, archi.copy()
    
    else :
        # compute size of the output of the last layer
        for l in feature_extra: 
            output_size = calcul_output(input_size, l)
            input_size = output_size
        
        # if we couldn't reduce more
        if(output_size == 1):
            return 0, archi.copy()
        
        # if the output size got more than 1 we can add new layer
        elif(output_size > 1): 
            output_size = calcul_output(output_size, layer)
            
            # if output size got negate is that the layer we want to add is wrong
            if(output_size < 1 ):
                return -1, archi.copy()
            # if output size is bigger than 0 we can add new layer and continue
            elif(output_size > 0):
                archi.append(layer)
                return 1, archi.copy()
            # this should not append
            else:
                return "Somethink wrong"
    # this should not append
    return "Error"

In [16]:
# test to understant padding

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D

model = Sequential(layers=[
    Conv2D(6, 7, input_shape=(28, 28, 1), strides=2, padding="valid"),
    MaxPool2D(pool_size=7, strides=5, padding="same")
])

for layer in model.layers:
    print(layer.output_shape)

Using TensorFlow backend.


(None, 11, 11, 6)
(None, 3, 3, 6)


In [17]:
# test for the fonction addLayer()
architecture = list()
architecture.append(InputLayer(shape=[29,29,1]))



code, architecture = addLayer(architecture.copy(), conv1)

print("code : " + str(code))

code, architecture = addLayer(architecture.copy(), conv1)

print("code : " + str(code))

code, architecture = addLayer(architecture.copy(), conv1)

print("code : " + str(code))

architecture

code : 1
code : 1
code : -1


[InputLayer(shape=[29, 29, 1]),
 Convolution(fct_activation='tanh', kernel=7, nb_filter=6, padding='valid', stride=2),
 Convolution(fct_activation='tanh', kernel=7, nb_filter=6, padding='valid', stride=2)]

# Creation architecture

In [18]:
extraction_feature= [Pooling, Convolution]

In [30]:
for i in range (1,11):
    architecture = list() # init list
    
    # add Input layer
    architecture.append(InputLayer(shape=[28,28,1]))
    
    nb_filter = 6 # init nb feature map
    
    # add convolution layer
    architecture.append(Convolution(
                kernel=kernel_value[random.randrange(5)], 
                padding=padding_value[random.randrange(2)], 
                stride=stride_value[random.randrange(5)], 
                nb_filter=nb_filter,
                fct_activation=fct_activation_value[random.randrange(3)]
    ))
    
    # add extraction feature (succession of Pooling/convolution)
    # Pooling can't be follow by a Pooling
    pooling = False
    code = 1 # 
    j = 2
    while(code == 1):
        layer = extraction_feature[random.randrange(2)]
        if(pooling | isinstance(layer, Convolution)):
            add_layer = Convolution(
                    kernel=kernel_value[random.randrange(5)], 
                    padding=padding_value[random.randrange(2)], 
                    stride=stride_value[random.randrange(5)], 
                    nb_filter=nb_filter*j,
                    fct_activation=fct_activation_value[random.randrange(3)]
            )
            j+=1
        else: 
            add_layer = Pooling(
                op = op_value[random.randrange(2)],
                kernel=kernel_value[random.randrange(5)], 
                padding=padding_value[random.randrange(2)], 
                stride=stride_value[random.randrange(5)]
            )
        code, architecture = addLayer(architecture, add_layer)
        if(code == 1 & isinstance(add_layer, Convolution)):
            pooling = False
        else: pooling = True
    
    if(not pooling):
        pooling = Pooling(
            op = op_value[random.randrange(2)],
            kernel=kernel_value[random.randrange(5)], 
            padding=padding_value[random.randrange(2)], 
            stride=stride_value[random.randrange(5)]
        )
        code, architecture = addLayer(architecture, pooling)
        while(code < 0):
            pooling = Pooling(
                    op = pooling.op,
                    kernel = pooling.kernel-1,
                    padding = pooling.padding,
                    stride = pooling.stride-1
                )
            code, architecture = addLayer(architecture, pooling)
        
    
    # add flatten Layer
    architecture.append(Flatten())
    
    # add Dense Layer
    nb_neurones = nb_neurones_value[random.randrange(5)]
    while((nb_neurones_value.index(nb_neurones)>0) & stop(len(architecture),4)):
        architecture.append(Dense(
            nb_neurones = nb_neurones,
            fct_activation = fct_activation_value[random.randrange(3)]      
        ))
        nb_neurones = nb_neurones_value[nb_neurones_value.index(nb_neurones)-1]
    
    architecture.append(Dense(
        nb_neurones = nb_neurones,
        fct_activation="softmax"           
    ))

    
                        
                        
    # print architecture
    print(architecture)
    
    # create file
    file_name = "archi_random_%d_v.json" % i
    print(file_name)
    print("")
    create_json_file(architecture, file_name)

[InputLayer(shape=[28, 28, 1]), Convolution(fct_activation='selu', kernel=3, nb_filter=6, padding='same', stride=5), Pooling(kernel=1, op='avg', padding='same', stride=4), Flatten(), Dense(fct_activation='softmax', nb_neurones=256)]
archi_random_1_v.json

[InputLayer(shape=[28, 28, 1]), Convolution(fct_activation='relu', kernel=5, nb_filter=6, padding='same', stride=5), Pooling(kernel=2, op='max', padding='valid', stride=1), Convolution(fct_activation='relu', kernel=3, nb_filter=12, padding='same', stride=4), Flatten(), Dense(fct_activation='tanh', nb_neurones=84), Dense(fct_activation='selu', nb_neurones=10), Dense(fct_activation='softmax', nb_neurones=1)]
archi_random_2_v.json

[InputLayer(shape=[28, 28, 1]), Convolution(fct_activation='selu', kernel=5, nb_filter=6, padding='valid', stride=1), Pooling(kernel=2, op='avg', padding='same', stride=4), Convolution(fct_activation='relu', kernel=1, nb_filter=12, padding='valid', stride=5), Flatten(), Dense(fct_activation='relu', nb_neurones